In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import preprocessing


from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
import nltk
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem import WordNetLemmatizer


from keras.optimizers import Adam
import os


from textblob import TextBlob

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train

In [ ]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Test ve Train datalarında location verisinde birçok değer eksik. Hedef sütunumuza bir göz atalım.

In [ ]:
train['target'].value_counts()

In [ ]:
sns.barplot(train['target'].value_counts().index,train['target'].value_counts(),palette='rocket')

Afet ve afet dışı tweetlerden bir örneği inceleyelim.

In [ ]:
disaster_tweets = train[train['target']==1]['text']
disaster_tweets.values[1]

In [ ]:
non_disaster_tweets = train[train['target']==0]['text']
non_disaster_tweets.values[1]

**Train ve Test verisi preprocessing**

train ve test verisi içerisindeki @<kullanici_adi> ibareleri temizleyelim.



In [ ]:

def remove_nick_name(text):
    
    return re.sub(r"\@\S+", "", text)

train['text']=train['text'].apply(lambda x : remove_nick_name(x))
test['text']=test['text'].apply(lambda x : remove_nick_name(x))
train[500:700]


train ve test verisi içerisindeki numaraları temizleyelim.

In [ ]:
def remove_numbers(self):
    result = ''.join(i for i in self if not i.isdigit())
    return(result)


train['text']=train['text'].apply(lambda x : remove_numbers(x))
test['text']=test['text'].apply(lambda x : remove_numbers(x))
train


train ve test verisi içerisindeki url temizleyelim.

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

train['text']=train['text'].apply(lambda x : remove_URL(x))
test['text']=test['text'].apply(lambda x : remove_URL(x))
test


train ve test verisi içerisindeki html temizleyelim.

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
train['text']=train['text'].apply(lambda x : remove_html(x))
test['text']=test['text'].apply(lambda x : remove_html(x))

train.head()


train ve test verisi içerisindeki emojileri temizleyelim.

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
train['text']=train['text'].apply(lambda x: remove_emoji(x))
test['text']=test['text'].apply(lambda x: remove_emoji(x))
train.head()


train ve test verisi içerisindeki noktalama işaretlerini temizleyelim.

In [ ]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

train['text']=train['text'].apply(lambda x : remove_punct(x))
test['text']=test['text'].apply(lambda x : remove_punct(x))
train.head()


Train ve Test verimizdeki bütün harfleri, küçük harf yapalım.

In [ ]:
train = train.applymap(lambda s:s.lower() if type(s) == str else s)
test = test.applymap(lambda s:s.lower() if type(s) == str else s)

train = train.applymap(lambda s:s.lower() if type(s) == str else s)
test = test.applymap(lambda s:s.lower() if type(s) == str else s)

In [ ]:
!pip install textblob

Text olarak verilen tweetlerin, sentiment(olumluluk/ olumsuzluk) durumlarını inceleyip, polarity sutunu oluşturup buraya kaydeder.

In [ ]:

def polarity_check_sayi(self):
    polarity_list = []
    for el in self['text']:
        result = TextBlob(el).sentiment.polarity
        polarity_list.append(result)

    self['polarity'] = polarity_list
    
    

def polarity_check_label(self):
    polarity_list = []
    for el in self['text']:
        result = TextBlob(el).sentiment.polarity
        if result < 0:
            check = 'Negative'
        elif result == 0:
            check = 'Neutral'
        else:
            check = 'Positive'
            
        polarity_list.append(check)

    self['polarity'] = polarity_list
    

    
polarity_check_label(test)
polarity_check_label(train)

test

Train ve Test datamızdaki  "keyword" ve "locations" sutünlarının her ikisinin de boş olduğu verileri silelim.

train.dropna(subset=['keyword', 'location'], how='all', inplace=True)
test.dropna(subset=['keyword', 'location'], how='all', inplace=True)
#test.dropna(inplace=True)

train

Veri setindeki locations sutununda en çok kullanılan değerleri görelim.

In [ ]:
loc_dict={'United States':'USA','New York':'USA',"London":'UK',"Los Angeles, CA":'USA',"Washington, D.C.":'USA',
          "California":'USA',"Chicago, IL":'USA',"Chicago":'USA',"New York, NY":'USA',"California, USA":'USA',
          "FLorida":'USA',"Nigeria":'Africa',"Kenya":'Africa',"Everywhere":'Worldwide',"San Francisco":'USA',
          "Florida":'USA',"United Kingdom":'UK',"Los Angeles":'USA',"Toronto":'Canada',"San Francisco, CA":'USA',
          "NYC":'USA',"Seattle":'USA',"Earth":'Worldwide',"Ireland":'UK',"London, England":'UK',"New York City":'USA',
          "Texas":'USA',"London, UK":'UK',"Atlanta, GA":'USA',"Mumbai":"India"}

train['location'].replace(loc_dict,inplace=True)

#Create barchart for top 10 locations using seaborn
sns.barplot(y=train['location'].value_counts()[:10].index,x=train['location'].value_counts()[:10],
            orient='h');

Location sütunumuzdaki boş olan verilere en çok kullanılan veriyi ekleyelim.

In [ ]:
train = train.replace(np.nan, 'usa', regex=True)
test = test.replace(np.nan, 'usa', regex=True)
train

Tokenization: Bir cümleyi kelimelerine ayırmaya yarar. Kelimeler bir liste oluşturur.

In [ ]:
text = "Benim adım Başak."
tokenizer1 = nltk.tokenize.WhitespaceTokenizer()
tokenizer2 = nltk.tokenize.TreebankWordTokenizer()
tokenizer3 = nltk.tokenize.WordPunctTokenizer()
tokenizer4 = nltk.tokenize.RegexpTokenizer(r'\w+')
print("-"*100)
print("Example Text: ",text)
print("-"*100)
print("Tokenization by whitespace:- ",tokenizer1.tokenize(text))
print("Tokenization by words using Treebank Word Tokenizer:- ",tokenizer2.tokenize(text))
print("Tokenization by punctuation:- ",tokenizer3.tokenize(text))
print("Tokenization by regular expression:- ",tokenizer4.tokenize(text))

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x))
test['text'] = test['text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
train['text'].head()

In [ ]:
test['text'].head()

İngilizce deki anlamsız kelimeleri çıkartalım "the gibi"

In [ ]:
def remove_stopwords(text):
    

    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [ ]:
train['text'] = train['text'].apply(lambda x : remove_stopwords(x))
test['text'] = test['text'].apply(lambda x : remove_stopwords(x))

In [ ]:
train.text.head()

In [ ]:
test.text.head()

Yukarıda dizin olarak ayrılan kelimeleri pre processing sonrası tekrar bir araya getirdik.

In [ ]:
def combine_text(text):
    
    '''
    Input-text= list cleand and tokenized text
    Output- Takes a list of text and returns combined one large chunk of text.
    
    '''
    all_text = ' '.join(text)
    return all_text
train['text'] = train['text'].apply(lambda x : combine_text(x))
test['text'] = test['text'].apply(lambda x : combine_text(x))
train.head()

In [ ]:
count_vectorizer=CountVectorizer()
train_cv=count_vectorizer.fit_transform(train["text"])
test_cv=count_vectorizer.transform(test["text"])
print(train_cv[0].todense())

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
train_tf=tfidf.fit_transform(train["text"])
test_tf=tfidf.transform(test["text"])

In [ ]:

#Split the CountVector vectorized data into train and test datasets for model training and testing
X_train_cv, X_test_cv, y_train_cv, y_test_cv =train_test_split(train_cv,train.target,test_size=0.2,random_state=2020)

In [ ]:
def fit_and_predict(model,X_train,y_train,X_test,y_test):
    
    '''Input- model=model to be trained
              X_train, y_train= traing data set
              X_test,  y_test = testing data set
       Output- Print accuracy of model for training and test data sets   
    '''
    
    # Fitting a simple Logistic Regression on Counts
    clf = model
    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    confusion_matrix(y_test,predictions)
    print(classification_report(y_test,predictions))
    print('-'*50)
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(clf.score(X_train, y_train)*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test,predictions)*100)))
    print('-'*50)

In [ ]:
models=[LogisticRegression(C=1.0),SVC(),MultinomialNB(),DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=5),RandomForestClassifier()]

In [ ]:

# Loop through the list of models and use 'fit_and_predict()' function to trian and make predictions
for model in models:
    fit_and_predict(model,X_train_cv, y_train_cv,X_test_cv,y_test_cv)

In [ ]:
# Split the TFDIF vectorized data into train and test datasets for model training and testing
X_train_tf, X_test_tf, y_train_tf, y_test_tf =train_test_split(train_tf,train.target,test_size=0.2,random_state=2020)

In [ ]:
# Loop through the list of models and use 'fit_and_predict()' function to train and make predictions on the TFDIF vectororized data
for model in models:
    fit_and_predict(model,X_train_tf, y_train_tf,X_test_tf,y_test_tf)

In [ ]:

# Printing model performance results.
results_dict={'Classifier':['Logistic regression', 'SVC', 'MultinomialNB', 'DecisionTreeClassifier',
                            'KNeighborsClassifier','RandomForestClassifier'],
              'F1-Score':[0.81, 0.40, .80, .75,0.65,0.76],'Accuracy':['81%', '56%', '80%','75%','69%','77%']} 
results=pd.DataFrame(results_dict)
results

In [ ]:

# Printing model performance results.
results_dict={'Classifier':['Logistic regression', 'SVC', 'MultinomialNB', 'DecisionTreeClassifier',
                            'KNeighborsClassifier','RandomForestClassifier'],
              'F1-Score':[0.81, 0.40, .80, .75,0.65,0.76],'Accuracy':['81%', '56%', '80%','75%','69%','77%']} 
results=pd.DataFrame(results_dict)
results

In [ ]:
#LogisticRegression(C=1.0)

In [ ]:
# Fitting 'LogisticRegression()' with CountVectorizer() fit dataset
clf_logreg = MultinomialNB()
clf_logreg.fit(X_train_cv, y_train_cv)
pred=clf_logreg.predict(X_test_cv)
confusion_matrix(y_test_cv,pred)
print(classification_report(y_test_cv,pred))
print('Accuracy of classifier on training set:{}%'.format(round(clf_logreg.score(X_train_cv, y_train_cv)*100)))
print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test_cv,pred)*100)))

In [ ]:
clf_logreg

sub_df = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sub_df["target"] = model.predict(test_cv)
sub_df.to_csv("submission.csv", index=False)


In [ ]:
sub_df = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sub_df["target"] = clf_logreg.predict(test_cv)
sub_df.to_csv("submission.csv", index=False)